# 라이브러리 및 데이터셋 불러오기

In [1]:
import pandas as pd
# # 컬럼 전체 보기
# pd.set_option('display.max_columns', None)

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 한글 폰트 설정
plt.rc('font', family='Malgun Gothic')  # 윈도우 기본 한글 폰트
plt.rc('axes', unicode_minus=False)     # 마이너스 부호 깨짐 방지

# 1. 데이터 셋 생성

## (1)내 플레이리스트 불러오기

In [2]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd
import time
import pprint

# 1. Spotify API 인증

client_id = ''
client_secret = ''
redirect_uri = ''


sp = spotipy.Spotify(
    auth_manager=SpotifyOAuth(
        client_id=client_id,
        client_secret=client_secret,
        redirect_uri=redirect_uri,
        scope="playlist-read-private user-library-read"
    )
)


In [3]:
# 2. 스포티파이 playlist 호출

# 2-1. 플레이리스트 1
mood_playlist_id = "6KGonoQ1ytIE18jWXNXjRK"
mood_results = sp.playlist_items(mood_playlist_id)
mood_tracks = mood_results['items']

while mood_results['next']:
    mood_results = sp.next(mood_results)
    mood_tracks.extend(mood_results['items'])
    print("추가됨 → 총 개수:", len(mood_tracks))
    
meta_list = []

for item in mood_tracks:
    track = item["track"]
    album = track["album"]
    artists = track["artists"]

    meta_list.append({
        "track_id": track["id"],
        "track_name": track["name"],
        "artist_name": ";".join([a["name"] for a in artists]),
        "album_name": album["name"],
        "album_release": album["release_date"],
        "duration_ms": track["duration_ms"],
        "explicit": track["explicit"],
        "popularity": track["popularity"]
    })

df_mood_meta = pd.DataFrame(meta_list)

추가됨 → 총 개수: 200
추가됨 → 총 개수: 300
추가됨 → 총 개수: 314


In [4]:
df_mood_meta

,track_id,track_name,artist_name,album_name,album_release,duration_ms,explicit,popularity
0,03YtYSkpHqqY3EBmHJkTjP,Till The End of Time,The Black Skirts,"Another Miss Oh, Pt. 7 (Original Television So...",2016-06-14,269000,False,55
1,3OOibIoFKsuR7V5j0dT8mW,Young Love,The Black Skirts,Don't You Worry Baby (I'm Only Swimming),2011-07-13,231946,False,35
2,7ePpQepOptZ1M9jRRydHsZ,별 보러 가자,Jukjae,FINE,2017-03-09,317531,False,52
3,797XRv2RhIp9qHeKG42HzD,"All Night Long (She is My Type♡ X Car, the gar...","Car, the garden","All Night Long (She is My Type♡ X Car, the gar...",2020-09-03,199746,False,47
4,3QYTZLJaVPuAYm7wr51Cor,"The Night, Gloomy Wind",O.WHEN,Tiny bits of life Part.11,2019-09-18,207360,False,19
...,...,...,...,...,...,...,...,...
309,2Ofqs4b2114JHEpkOoCfNE,josee!,데이먼스 이어 Damons year,josee!,2018-08-17,123913,False,56
310,41nIJErYY9goKvnCyG7ov6,The Frost,Mitski,The Land Is Inhospitable and So Are We,2023-09-15,168480,False,57
311,62JBtqVL1qQO3SE3JuFimS,Salty,데이먼스 이어 Damons year,“ Sin ! ”,2019-10-29,176014,False,55
312,1llrs3s4loOC4eMaV1yc1x,"Flicker (Feat. Car, the garden)","youra;Car, the garden",B side,2019-03-05,197750,False,37


## (2) 곡 오디오 피쳐 가져오기

스포티파이에서는 2024년도부터 오디오 피쳐 제공을 하지 않는다.

이를 대체하기 위해 soundcharts에서 제공하는 오디오 피쳐를 사용할 예정.

soundcharts에서는 스포티파이의 곡ID를 토대로 오디오 피쳐를 제공한다.

In [5]:
from soundcharts.client import SoundchartsClient
import requests


#내 api 계정
headers = {
    'x-app-id': 'KJUNSEOK-API_E4BDD206',
    'x-api-key': '0f25550106a1b358',
}


In [6]:
# 오디오 피쳐 가져오기 함수
def get_song_features(platform, track_id):
    url = f'https://customer.api.soundcharts.com/api/v2.25/song/by-platform/{platform}/{track_id}'
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        song = data.get("song", data.get("object", {}))
        audio_features = song.get("audio", {})

        # 장르 안전 처리
        genres_list = song.get("genres", [])
        if genres_list:  # 리스트가 비었는지 확인
            genres = genres_list[0]
            root_str = genres.get("root") or ""
            main_genres_list = root_str.split(",")
            main_genres = main_genres_list[0].strip() if main_genres_list else None

            sub_list = genres.get("sub") or []
            sub_genres = sub_list[0] if sub_list else None
        else:
            main_genres = None
            sub_genres = None

        result = {
            "track_id": track_id,
            "title": song.get("name"),
            "artist": ", ".join([a.get("name") for a in song.get("artists", [])]),
            "duration": song.get("duration"),
            "main_genres": main_genres,
            "sub_genres": sub_genres,
            "releaseDate": song.get("releaseDate"),
            "explicit": song.get("explicit"),
            **audio_features
        }
        return result

    else:
        print(f"Error {response.status_code} for track {track_id}")
        return None


In [7]:
# 오디오 피쳐 가져오기
track_ids = df_mood_meta['track_id'].tolist()

all_data = []

# 곡 정보 모두 리스트화
for tid in track_ids:
  features = get_song_features("spotify", tid)
  all_data.append(features)

# 데이터 프레임으로 변환
clean_data = [{ } if v is None else v for v in all_data]
df = pd.DataFrame(clean_data)

In [9]:
# popularity 병합처리
df_final = df.merge(df_mood_meta[['track_id', 'popularity']], on='track_id', how='left')
df_final.to_csv('dataset.csv', index=False, encoding="utf-8-sig")


In [10]:
df_final

,track_id,title,artist,duration,main_genres,sub_genres,releaseDate,explicit,acousticness,danceability,...,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,timeSignature,valence,popularity
0,03YtYSkpHqqY3EBmHJkTjP,Till the End of Time,The Black Skirts,269,pop,k-pop,2016-06-14T00:00:00+00:00,False,0.87,0.55,...,0.23,0,0.13,-12.23,1,0.03,120.11,4,0.20,55
1,3OOibIoFKsuR7V5j0dT8mW,젊은 우리 사랑,The Black Skirts,231,asian,asian,2011-07-13T00:00:00+00:00,False,0.22,0.72,...,0.00,0,0.14,-7.88,1,0.04,94.04,4,0.68,35
2,7ePpQepOptZ1M9jRRydHsZ,별 보러 가자,Jukjae,317,asian,asian,2017-03-09T00:00:00+00:00,False,0.63,0.53,...,0.00,5,0.11,-10.00,0,0.03,66.94,3,0.31,52
3,797XRv2RhIp9qHeKG42HzD,"All Night Long (She is My Type♡ X Car, the gar...","Car, the Garden",199,None,None,2020-09-03T00:00:00+00:00,False,0.39,0.64,...,0.00,3,0.09,-4.99,1,0.02,95.04,4,0.21,47
4,3QYTZLJaVPuAYm7wr51Cor,"The Night, Gloomy Wind",O.WHEN,207,alternative,alternative,2019-09-18T00:00:00+00:00,False,0.86,0.39,...,0.00,3,0.10,-6.16,0,0.03,102.00,3,0.22,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309,2Ofqs4b2114JHEpkOoCfNE,Josee!,데이먼스 이어 Damons year,123,asian,asian,2018-08-17T00:00:00+00:00,False,0.92,0.41,...,0.00,10,0.11,-8.63,1,0.04,151.48,4,0.25,56
310,41nIJErYY9goKvnCyG7ov6,The Frost,Mitski,168,alternative,alternative,2023-09-15T00:00:00+00:00,False,0.90,0.46,...,0.03,10,0.30,-17.20,1,0.03,106.94,4,0.35,57
311,62JBtqVL1qQO3SE3JuFimS,salty,데이먼스 이어 Damons year,176,asian,asian,2019-10-29T00:00:00+00:00,False,0.90,0.58,...,0.00,11,0.11,-8.35,1,0.04,126.33,4,0.53,55
312,1llrs3s4loOC4eMaV1yc1x,Flicker,"Car, the Garden, youra",197,asian,asian,2019-03-05T00:00:00+00:00,False,0.93,0.68,...,0.00,9,0.08,-6.56,0,0.05,78.85,3,0.55,37
